In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-187048
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-187048


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RB2KEXL8N to authenticate.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "supercluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.01, 0.1, 1, 10, 100),
        '--max_iter' : choice(10, 50, 100, 200)

    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_cluster,
    environment=sklearn_env,
)

# # submit script to AML
# run = exp.submit(config)
# print(run.get_portal_url()) # link to ml.azure.com
# run.wait_for_completion(show_output=True)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    run_config=src,
    max_total_runs=16,
    max_concurrent_runs=4
)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_3047b25d-892e-4294-84c7-e11b733fab9d
Web View: https://ml.azure.com/runs/HD_3047b25d-892e-4294-84c7-e11b733fab9d?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-187048/workspaces/quick-starts-ws-187048&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-18T19:50:50.108923][API][INFO]Experiment created<END>\n""<START>[2022-02-18T19:50:50.849648][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-18T19:50:51.623754][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_3047b25d-892e-4294-84c7-e11b733fab9d
Web View: https://ml.azure.com/runs/HD_3047b25d-892e-4294-84c7-e11b733fab9d?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-187048/workspaces/quick-starts-ws-187048&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_3047b25d-892e-4294-84c7-e11b733fab9d',
 'target': 'supercluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-18T19:50:49.873825Z',
 'endTimeUtc': '2022-02-18T20:11:36.683479Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '15a93251-a0fd-422b-856d-57bacf224790',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '20',
  'score': '0.9116240573845871',
  'best_child_run_id': 'HD_3047b25d-892e-4294-84c7-e11b733fab9d_5',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187048.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3047b25d-892e-4294-84c7-e11b733fab9d/azureml-logs/hyp

In [8]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print('Best run metrics: ', best_run.get_metrics())
print('Best run details ', best_run.get_details())


Best run metrics:  {'Regularization Strength:': 0.01, 'Max iterations:': 50, 'Accuracy': 0.9116240573845871}
Best run details  {'runId': 'HD_3047b25d-892e-4294-84c7-e11b733fab9d_5', 'target': 'supercluster', 'status': 'Completed', 'startTimeUtc': '2022-02-18T20:06:13.795148Z', 'endTimeUtc': '2022-02-18T20:06:17.43558Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '15a93251-a0fd-422b-856d-57bacf224790', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.01', '--max_iter', '50'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'supercluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceType

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url_path = ["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"]

ds = TabularDatasetFactory.from_delimited_files(url_path)
### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=10,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###